In [1]:
import os
import base64
import requests
import pandas as pd
from io import StringIO
from google.colab import userdata

In [13]:
api_key = userdata.get('API-key')

In [14]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def explain_image_base64(question, base64_image):

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": question
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    openai_url = "https://api.openai.com/v1/chat/completions"

    response = requests.post(openai_url, headers=headers, json=payload)
    return response.json()

def explain_image(question, image_path):
    base64_image = encode_image(image_path)
    return explain_image_base64(question, base64_image)

In [16]:
image_path = r"Image.jpg"
json_answer = explain_image("""
Create csv file from this image with column 3 columns.
In first column 'Latin Content' collect all about Latin content.
In second column 'Gender' place gender of word (m/f/n).
In last column 'German Translations' collect transtaltions.
Be aware that some rows are places in 2 lines - please combine it correctly based on gray line seperators.
User separator |. Be sure that csv has exactly 3 columns.
""", image_path)
answer = json_answer['choices'][0]['message']['content']
print(answer)

Latin Content|Gender|German Translations
potentia|f|die Macht
instituere, instituō, instituī, institūtum|n|beginnen, einrichten, unterrichten
prōspicere, prōspiciō, prōspexī, prōspectum|m|achtgeben, dafür sorgen, (vor)sehen
mortem prōspicere|f|den Tod vorhersehen
salūtī patris prōspicere|f|für das Wohl des Vaters sorgen
pater familiās|m|der Hauser, das Familienoberhaupt
inīustus, a, um|m|ungerecht
ōrdō, ōrdinis|m|die Ordnung, die Reihe, der Stand
ōrdō senātōrum|m|der Senatorenstand
eiusdem ōrdinis|m|von gleichem Rang (Stand)
nōbilitās, nōbilitātis|f|der Adel, die vornehme Abstammung
aetās, aetātis|f|das Lebensalter, die Zeit, das Zeitalter
-ve||oder
subicere, subiciō, subiēcī, subiectum|m|darunterlegen; unterwerfen
parcere hostibus subiectīs||die unterworfenen Feinde schonen
rēs familiāris|f|das (Privat-)Vermögen, der Besitz
eripere, ēripiō, ēripuī, ēreptum|m|entreißen
eicere, eiciō, eiēcī, eiectum|m|hinauswerfen, vertreiben
reliquus, a, um|m|übrig, künftig
plūris esse||teuer sein
nequ

In [17]:
lines = answer.split('\n')
lines

['Latin Content|Gender|German Translations',
 'potentia|f|die Macht',
 'instituere, instituō, instituī, institūtum|n|beginnen, einrichten, unterrichten',
 'prōspicere, prōspiciō, prōspexī, prōspectum|m|achtgeben, dafür sorgen, (vor)sehen',
 'mortem prōspicere|f|den Tod vorhersehen',
 'salūtī patris prōspicere|f|für das Wohl des Vaters sorgen',
 'pater familiās|m|der Hauser, das Familienoberhaupt',
 'inīustus, a, um|m|ungerecht',
 'ōrdō, ōrdinis|m|die Ordnung, die Reihe, der Stand',
 'ōrdō senātōrum|m|der Senatorenstand',
 'eiusdem ōrdinis|m|von gleichem Rang (Stand)',
 'nōbilitās, nōbilitātis|f|der Adel, die vornehme Abstammung',
 'aetās, aetātis|f|das Lebensalter, die Zeit, das Zeitalter',
 '-ve||oder',
 'subicere, subiciō, subiēcī, subiectum|m|darunterlegen; unterwerfen',
 'parcere hostibus subiectīs||die unterworfenen Feinde schonen',
 'rēs familiāris|f|das (Privat-)Vermögen, der Besitz',
 'eripere, ēripiō, ēripuī, ēreptum|m|entreißen',
 'eicere, eiciō, eiēcī, eiectum|m|hinauswerfen

In [18]:
df = pd.read_csv(StringIO(answer), delimiter='|', names = ['L', 'G', 'D'], header = 0)

In [19]:
df

,L,G,D
0,potentia,f,die Macht
1,"instituere, instituō, instituī, institūtum",n,"beginnen, einrichten, unterrichten"
2,"prōspicere, prōspiciō, prōspexī, prōspectum",m,"achtgeben, dafür sorgen, (vor)sehen"
3,mortem prōspicere,f,den Tod vorhersehen
4,salūtī patris prōspicere,f,für das Wohl des Vaters sorgen
5,pater familiās,m,"der Hauser, das Familienoberhaupt"
6,"inīustus, a, um",m,ungerecht
7,"ōrdō, ōrdinis",m,"die Ordnung, die Reihe, der Stand"
8,ōrdō senātōrum,m,der Senatorenstand
9,eiusdem ōrdinis,m,von gleichem Rang (Stand)


In [21]:
df['result'] = df.apply(lambda row:
    f'{row["L"]}<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">{row["G"]}</span></i>|{row["D"]}<br><span style="color: rgb(170, 170, 0);">',
    axis=1)



In [22]:
result_str = '\n'.join(df['result'])

In [23]:
print(result_str)

potentia<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">f</span></i>|die Macht<br><span style="color: rgb(170, 170, 0);">
instituere, instituō, instituī, institūtum<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">n</span></i>|beginnen, einrichten, unterrichten<br><span style="color: rgb(170, 170, 0);">
prōspicere, prōspiciō, prōspexī, prōspectum<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m</span></i>|achtgeben, dafür sorgen, (vor)sehen<br><span style="color: rgb(170, 170, 0);">
mortem prōspicere<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">f</span></i>|den Tod vorhersehen<br><span style="color: rgb(170, 170, 0);">
salūtī patris prōspicere<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">f</span></i>|für das Wohl des Vaters sorgen<br><span style="color: rgb(170, 170, 0);">
pater familiās<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m</span></i>|der Hauser, das Familienoberhaupt<br><span style="color: rgb(170, 170, 0);">
in